In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
# from simpletransformers.classification import ClassificationModel
# from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import ast
import numpy as np
device = torch.device("cuda")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# from google.colab import files
# uploaded = files.upload()

In [5]:
multi_label_classification = pd.read_csv("/content/multilabel_classification", usecols=["text", "labels"])
multi_label_classification.head()

,text,labels
0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...


In [0]:
# summary_state = summary_state.drop(columns=["State", "Count"])
# summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
# summary_state = summary_state.reset_index(drop=True)
multi_label_classification = multi_label_classification.reset_index()

In [7]:
separation = int(0.8 * len(multi_label_classification))
train_data = multi_label_classification[:separation]
test_data = multi_label_classification[separation:]
print(len(test_data))
train_data

32


,index,text,labels
0,0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...
...,...,...,...
123,123,From week 40 to 43 (corresponding to October 2...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. ...
124,124,From week 44 to 47 (corresponding to November ...,[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. ...
125,125,From week 48 to 52 (corresponding to December ...,[0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. ...
126,126,The first week of the year 2017 is marked by t...,[0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. ...


In [8]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hs6pqpqj
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-hs6pqpqj
     |████████████████████████████████| 3.8MB 3.5MB/s 
     |████████████████████████████████| 1.1MB 40.1MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=666257 sha256=edc32ad764ec652ec690e021da6b645bfab027cf3726ddcb488a203c12dba5bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-mmfzity8/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0e28b2aa3b7b659fbc34f94586c5f0d811614d73f86ddc2f7a1903055e41bbae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [0]:
from transformers import LongformerModel, LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
model.cuda()

LongformerModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BertSelfOutpu

In [0]:
torch.cuda.empty_cache()
#note when encoded the start of every token is added with an <s> for classification purposes
input_ids = tokenizer.encode("Hello, my dog is cute")  # Batch size 1
print(tokenizer.decode(input_ids))
print(tokenizer.decode(input_ids[0]))

Thu Jun  4 07:40:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    32W /  70W |   1515MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
tokenized = multi_label_classification["text"]\
                                    .apply(lambda x: tokenizer.encode(x, add_special_tokens_tokens=True))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)
#padded according to max_length so we can feed the model in batches - padding works as such, add 0s after the length
#of the original sequence l such that the new length is = max_len = 2530
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


2530


In [16]:
np.array(padded.shape)
#Add masking so that know BERT knows where to attend to => so it can ignore padding
#adds a 1 everwhere where the condition of padded != 0 is satisfied, in other words everywhere there isn't padding
attention_mask = np.where(padded != 0, 1, 0)
#add global attention represented with 2 - for now just add 1 at the classification <s> token
attention_mask[:, [0, -1]] = 2
print(attention_mask)
device = torch.device("cuda")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)

def create_save_feature_representation():
  first_time = True
  for i in range(0,len(input_ids),2):
    with torch.no_grad():
      predictions, hidden_states = model(input_ids[i:i+2], attention_mask=attention_mask[i:i+2])
      scores = predictions[:,0,:].cpu().numpy()
      if not first_time:
        curr = np.load("lang_features.npy")
        scores = np.concatenate((curr, scores))
      else:
        first_time = False
      np.save("lang_features.npy",scores)

# create_save_feature_representation()

NameError: ignored

In [0]:
features = np.load("lang_features.npy")
#convert the labels into numpy arrays since they are stored as string representations
multi_label_classification["labels"] = multi_label_classification["labels"].apply(lambda x: np.fromstring(x[1:-1], dtype=np.float32, sep=' '))

In [0]:
train_features, test_features = features[:separation], features[separation:]
hidden_size = features.shape[1]

#convert to float datatype that can be then cast to a tensor in PyTorch
labels = np.array([label for label in np.array(multi_label_classification["labels"].values)])
train_labels, test_labels = labels[:separation], labels[separation:]

In [33]:
print(train_labels.dtype)
epochs = 50
neural_network = nn.Sequential(
                    nn.Linear(hidden_size, 1000),
                    nn.ReLU(),
                    nn.BatchNorm1d(1000),
                    nn.Linear(1000, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 600),
                    nn.ReLU(),
                    nn.BatchNorm1d(600),
                    nn.Linear(600, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 200),
                    nn.ReLU(),
                    nn.BatchNorm1d(200),
                    nn.Linear(200, 20),
                    nn.Sigmoid(),
                    )
neural_network.cuda()
dtype = torch.cuda.FloatTensor
neural_network.type = torch.cuda.FloatTensor
train_features, test_features = torch.tensor(train_features).to(device), torch.tensor(test_features).to(device)
train_labels, test_labels = torch.tensor(train_labels).to(device), torch.tensor(test_labels).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(neural_network.parameters(), lr=0.0001)

for epoch in range(epochs):
  optimizer.zero_grad()
  output = neural_network(train_features)
  loss = criterion(output, train_labels)
  print("Epoch {0} with training loss: {1}".format(epoch, loss))
  loss.backward()
  optimizer.step()    

torch.float32
Epoch 0 with training loss: 0.8404632806777954
Epoch 1 with training loss: 0.8198241591453552
Epoch 2 with training loss: 0.8078593015670776
Epoch 3 with training loss: 0.7995196580886841
Epoch 4 with training loss: 0.7932328581809998
Epoch 5 with training loss: 0.7881842255592346
Epoch 6 with training loss: 0.7840879559516907
Epoch 7 with training loss: 0.7806599736213684
Epoch 8 with training loss: 0.7776054739952087
Epoch 9 with training loss: 0.774965226650238
Epoch 10 with training loss: 0.772618293762207
Epoch 11 with training loss: 0.7705239057540894
Epoch 12 with training loss: 0.768570601940155
Epoch 13 with training loss: 0.7667591571807861


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 14 with training loss: 0.7651485800743103
Epoch 15 with training loss: 0.763664186000824
Epoch 16 with training loss: 0.76230788230896
Epoch 17 with training loss: 0.7610608339309692
Epoch 18 with training loss: 0.7599023580551147
Epoch 19 with training loss: 0.7588008046150208
Epoch 20 with training loss: 0.7577433586120605
Epoch 21 with training loss: 0.7567439079284668
Epoch 22 with training loss: 0.7558051347732544
Epoch 23 with training loss: 0.7549123764038086
Epoch 24 with training loss: 0.7540607452392578
Epoch 25 with training loss: 0.7532213926315308
Epoch 26 with training loss: 0.7524320483207703
Epoch 27 with training loss: 0.751680314540863
Epoch 28 with training loss: 0.7509732842445374
Epoch 29 with training loss: 0.7503077983856201
Epoch 30 with training loss: 0.7496556043624878
Epoch 31 with training loss: 0.7490279674530029
Epoch 32 with training loss: 0.7484250068664551
Epoch 33 with training loss: 0.7478465437889099
Epoch 34 with training loss: 0.7472730875015

In [37]:
def convert_to_thresholds(data):
  return data>0.5

def test_model():
  preds = neural_network(test_features).cpu().data.numpy()
  correct = test_labels.cpu().numpy()
  preds = convert_to_thresholds(preds)
  print("F1 Score: ", f1_score(preds, correct, average="micro"))
  print("ROC-AUC: ", roc_auc_score(correct, preds))

test_model()

F1 Score:  0.5296950240770466
ROC-AUC:  0.5406322402000238
